In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib
import warnings
warnings.filterwarnings('ignore')

class POWERGRIDDecisionSystem:
    def __init__(self):
        self.model = None
        self.label_encoders = {}
        self.feature_columns = []
        self.optimal_ranges = {}
        
    def load_data_and_train(self):
        """Load the dataset and train the model"""
        print("📊 Loading POWERGRID Dataset...")
        df = pd.read_csv('powergrid_demand_forecast_dataset.csv')
        
        # Clean and prepare data
        df_cleaned = self.clean_data(df)
        
        # Feature engineering
        df_processed, self.label_encoders = self.engineer_features(df_cleaned)
        
        # Define features
        self.feature_columns = [
            'Budget_Cr', 'Tax_Rate', 'Geographic_Risk', 
            'Month_Sin', 'Month_Cos', 
            'Budget_Risk_Interaction', 'Tax_Budget_Ratio', 'Risk_Adjusted_Budget',
            'State_Encoded', 'Tower_Type_Encoded', 'Substation_Type_Encoded'
        ]
        
        X = df_processed[self.feature_columns]
        y = df_processed['Material_Demand_Tons']
        
        # Train model
        print("🤖 Training Decision Model...")
        self.model = RandomForestRegressor(
            n_estimators=200,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        self.model.fit(X, y)
        
        # Calculate optimal ranges from historical data
        self.calculate_optimal_ranges(df_processed)
        
        print("✅ Decision System Ready!")
        return self.model
    
    def clean_data(self, df):
        """Clean and prepare the dataset"""
        df_clean = df.copy()
        
        # Convert Month to numeric if needed
        if df_clean['Month'].dtype == 'object':
            month_map = {
                'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
            }
            df_clean['Month'] = df_clean['Month'].map(month_map)
        
        # Ensure numeric types
        numeric_cols = ['Budget_Cr', 'Tax_Rate', 'Geographic_Risk', 'Month', 'Material_Demand_Tons']
        for col in numeric_cols:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        
        return df_clean.dropna()
    
    def engineer_features(self, df):
        """Create advanced features for better predictions"""
        df_processed = df.copy()
        
        # Cyclical features
        df_processed['Month_Sin'] = np.sin(2 * np.pi * df_processed['Month'] / 12)
        df_processed['Month_Cos'] = np.cos(2 * np.pi * df_processed['Month'] / 12)
        
        # Interaction features
        df_processed['Budget_Risk_Interaction'] = df_processed['Budget_Cr'] * df_processed['Geographic_Risk']
        df_processed['Tax_Budget_Ratio'] = df_processed['Tax_Rate'] / df_processed['Budget_Cr']
        df_processed['Risk_Adjusted_Budget'] = df_processed['Budget_Cr'] * (1 + df_processed['Geographic_Risk'] / 10)
        
        # Encode categorical variables
        categorical_columns = ['State', 'Tower_Type', 'Substation_Type']
        label_encoders = {}
        
        for col in categorical_columns:
            le = LabelEncoder()
            df_processed[col + '_Encoded'] = le.fit_transform(df_processed[col])
            label_encoders[col] = le
            
        return df_processed, label_encoders
    
    def calculate_optimal_ranges(self, df):
        """Calculate optimal parameter ranges from successful projects"""
        # Define success criteria (projects with optimal cost-to-demand ratio)
        df['cost_efficiency'] = df['Material_Demand_Tons'] / df['Budget_Cr']
        optimal_threshold = df['cost_efficiency'].quantile(0.75)  # Top 25% efficient projects
        
        successful_projects = df[df['cost_efficiency'] >= optimal_threshold]
        
        self.optimal_ranges = {
            'Budget_Cr': {
                'min': successful_projects['Budget_Cr'].min(),
                'max': successful_projects['Budget_Cr'].max(),
                'optimal': successful_projects['Budget_Cr'].median()
            },
            'Geographic_Risk': {
                'min': successful_projects['Geographic_Risk'].min(),
                'max': successful_projects['Geographic_Risk'].max(),
                'optimal': successful_projects['Geographic_Risk'].median()
            },
            'Month': {
                'optimal_months': [1, 2, 3, 10, 11]  # Best construction months
            }
        }
    
    def predict_demand(self, input_features):
        """Predict material demand for given inputs"""
        features = self.prepare_features(input_features)
        feature_array = np.array([features[col] for col in self.feature_columns]).reshape(1, -1)
        prediction = self.model.predict(feature_array)[0]
        return prediction
    
    def prepare_features(self, input_features):
        """Prepare features for prediction"""
        features = input_features.copy()
        
        # Ensure proper data types
        features['Budget_Cr'] = float(features['Budget_Cr'])
        features['Tax_Rate'] = float(features['Tax_Rate'])
        features['Geographic_Risk'] = float(features['Geographic_Risk'])
        features['Month'] = int(features['Month'])
        
        # Feature engineering
        features['Month_Sin'] = np.sin(2 * np.pi * features['Month'] / 12)
        features['Month_Cos'] = np.cos(2 * np.pi * features['Month'] / 12)
        features['Budget_Risk_Interaction'] = features['Budget_Cr'] * features['Geographic_Risk']
        features['Tax_Budget_Ratio'] = features['Tax_Rate'] / features['Budget_Cr']
        features['Risk_Adjusted_Budget'] = features['Budget_Cr'] * (1 + features['Geographic_Risk'] / 10)
        
        # Encode categorical variables
        for col in ['State', 'Tower_Type', 'Substation_Type']:
            le = self.label_encoders[col]
            if features[col] in le.classes_:
                features[col + '_Encoded'] = le.transform([features[col]])[0]
            else:
                features[col + '_Encoded'] = 0
                
        return features
    
    def analyze_optimization(self, input_features):
        """Analyze how to optimize the current project parameters"""
        base_prediction = self.predict_demand(input_features)
        recommendations = []
        
        # Test different scenarios
        scenarios = self.generate_optimization_scenarios(input_features)
        
        for scenario_name, scenario in scenarios.items():
            scenario_prediction = self.predict_demand(scenario)
            cost_efficiency = scenario_prediction / scenario['Budget_Cr']
            
            recommendations.append({
                'scenario': scenario_name,
                'predicted_demand': scenario_prediction,
                'budget': scenario['Budget_Cr'],
                'cost_efficiency': cost_efficiency,
                'improvement': ((base_prediction / input_features['Budget_Cr']) - cost_efficiency) / (base_prediction / input_features['Budget_Cr']) * 100
            })
        
        return sorted(recommendations, key=lambda x: x['cost_efficiency'], reverse=True)
    
    def generate_optimization_scenarios(self, base_features):
        """Generate alternative scenarios for optimization"""
        scenarios = {}
        
        # Scenario 1: Optimal budget adjustment
        scenarios['budget_optimized'] = base_features.copy()
        scenarios['budget_optimized']['Budget_Cr'] = self.optimal_ranges['Budget_Cr']['optimal']
        
        # Scenario 2: Better timing
        scenarios['timing_optimized'] = base_features.copy()
        scenarios['timing_optimized']['Month'] = 2  # Optimal construction month
        
        # Scenario 3: Risk mitigation
        scenarios['risk_optimized'] = base_features.copy()
        scenarios['risk_optimized']['Geographic_Risk'] = max(3, base_features['Geographic_Risk'] - 2)
        
        # Scenario 4: Combined optimization
        scenarios['fully_optimized'] = base_features.copy()
        scenarios['fully_optimized']['Budget_Cr'] = self.optimal_ranges['Budget_Cr']['optimal']
        scenarios['fully_optimized']['Month'] = 2
        scenarios['fully_optimized']['Geographic_Risk'] = max(3, base_features['Geographic_Risk'] - 1)
        
        return scenarios
    
    def get_decision_recommendations(self, input_features):
        """Generate comprehensive decision recommendations"""
        base_demand = self.predict_demand(input_features)
        optimization_analysis = self.analyze_optimization(input_features)
        
        recommendations = {
            'current_prediction': base_demand,
            'current_efficiency': base_demand / input_features['Budget_Cr'],
            'optimization_options': optimization_analysis,
            'risk_assessment': self.assess_risks(input_features),
            'procurement_strategy': self.suggest_procurement_strategy(input_features, base_demand)
        }
        
        return recommendations
    
    def assess_risks(self, input_features):
        """Assess project risks based on inputs"""
        risks = []
        
        # High geographic risk
        if input_features['Geographic_Risk'] >= 7:
            risks.append({
                'type': 'High Geographic Risk',
                'level': 'High',
                'impact': '15-20% cost overrun possible',
                'mitigation': 'Increase safety stock by 15%'
            })
        
        # Suboptimal timing
        if input_features['Month'] in [6, 7, 8]:  # Monsoon months
            risks.append({
                'type': 'Monsoon Construction',
                'level': 'Medium',
                'impact': 'Potential 30% delay in construction',
                'mitigation': 'Plan indoor pre-fabrication work'
            })
        
        # Budget constraints
        avg_budget = self.optimal_ranges['Budget_Cr']['optimal']
        if input_features['Budget_Cr'] < avg_budget * 0.8:
            risks.append({
                'type': 'Budget Constraint',
                'level': 'High',
                'impact': 'Risk of material shortages',
                'mitigation': 'Consider phased procurement'
            })
            
        return risks
    
    def suggest_procurement_strategy(self, input_features, predicted_demand):
        """Suggest optimal procurement strategy"""
        strategy = {}
        
        # Safety stock calculation
        if input_features['Geographic_Risk'] >= 7:
            safety_margin = 0.15  # 15% for high risk areas
        else:
            safety_margin = 0.10  # 10% for normal areas
            
        strategy['recommended_procurement'] = predicted_demand * (1 + safety_margin)
        strategy['safety_margin'] = safety_margin * 100
        
        # Procurement timing
        if input_features['Month'] in [1, 2, 3]:  # Peak season
            strategy['timing'] = 'Immediate procurement recommended'
            strategy['lead_time'] = '4-6 weeks'
        else:
            strategy['timing'] = 'Can schedule procurement 2-3 months in advance'
            strategy['lead_time'] = '8-10 weeks'
            
        # Vendor strategy
        if predicted_demand > 1500:  # Large projects
            strategy['vendor_approach'] = 'Multiple vendors recommended for bulk materials'
        else:
            strategy['vendor_approach'] = 'Single reliable vendor sufficient'
            
        return strategy

# Initialize the decision system
decision_system = POWERGRIDDecisionSystem()
decision_system.load_data_and_train()

print("🚀 POWERGRID Real-Time Decision System Ready!")
print("=" * 60)

📊 Loading POWERGRID Dataset...
🤖 Training Decision Model...
✅ Decision System Ready!
🚀 POWERGRID Real-Time Decision System Ready!


In [3]:
def interactive_decision_maker():
    """Interactive interface for real-time decision making"""
    print("🏗️ POWERGRID REAL-TIME DECISION MAKER")
    print("=" * 50)
    
    while True:
        print("\n📝 Enter Project Details (or 'quit' to exit):")
        
        # Collect real-time inputs
        state = input("State: ").strip()
        if state.lower() == 'quit':
            break
            
        tower_type = input("Tower Type (Type-A/Type-B/Type-C/Type-D): ").strip()
        substation_type = input("Substation Type (132kV/220kV/400kV/765kV): ").strip()
        
        try:
            budget = float(input("Budget (Crores): "))
            tax_rate = float(input("Tax Rate (%): "))
            month = int(input("Month (1-12): "))
            geo_risk = int(input("Geographic Risk (1-10): "))
        except ValueError:
            print("❌ Invalid input! Please enter numeric values.")
            continue
        
        # Prepare input features
        input_features = {
            'State': state,
            'Tower_Type': tower_type,
            'Substation_Type': substation_type,
            'Budget_Cr': budget,
            'Tax_Rate': tax_rate,
            'Month': month,
            'Geographic_Risk': geo_risk
        }
        
        # Get comprehensive recommendations
        recommendations = decision_system.get_decision_recommendations(input_features)
        
        # Display results
        display_decision_results(input_features, recommendations)
        
        # Ask if user wants another analysis
        continue_analysis = input("\n🔍 Perform another analysis? (y/n): ").strip().lower()
        if continue_analysis != 'y':
            break

def display_decision_results(input_features, recommendations):
    """Display comprehensive decision results"""
    print("\n" + "=" * 60)
    print("🎯 DECISION ANALYSIS RESULTS")
    print("=" * 60)
    
    print(f"\n📊 PROJECT OVERVIEW:")
    print(f"   Location: {input_features['State']}")
    print(f"   Configuration: {input_features['Tower_Type']} + {input_features['Substation_Type']}")
    print(f"   Budget: {input_features['Budget_Cr']} Cr | Risk: {input_features['Geographic_Risk']}/10")
    print(f"   Timeline: Month {input_features['Month']}")
    
    print(f"\n📦 MATERIAL PREDICTION:")
    print(f"   Base Demand: {recommendations['current_prediction']:.0f} tons")
    print(f"   Cost Efficiency: {recommendations['current_efficiency']:.2f} tons/Cr")
    
    print(f"\n💡 OPTIMIZATION OPPORTUNITIES:")
    for i, option in enumerate(recommendations['optimization_options'][:3], 1):
        print(f"   {i}. {option['scenario'].replace('_', ' ').title()}:")
        print(f"      Demand: {option['predicted_demand']:.0f} tons | Budget: {option['budget']} Cr")
        print(f"      Efficiency: {option['cost_efficiency']:.2f} tons/Cr")
        print(f"      Improvement: {option['improvement']:+.1f}%")
    
    print(f"\n⚠️  RISK ASSESSMENT:")
    if recommendations['risk_assessment']:
        for risk in recommendations['risk_assessment']:
            print(f"   🔴 {risk['type']} ({risk['level']}): {risk['impact']}")
            print(f"      💡 Mitigation: {risk['mitigation']}")
    else:
        print("   ✅ Low risk profile detected")
    
    print(f"\n🛒 PROCUREMENT STRATEGY:")
    strategy = recommendations['procurement_strategy']
    print(f"   Recommended Quantity: {strategy['recommended_procurement']:.0f} tons")
    print(f"   Safety Margin: {strategy['safety_margin']}%")
    print(f"   Timing: {strategy['timing']}")
    print(f"   Lead Time: {strategy['lead_time']}")
    print(f"   Vendor Approach: {strategy['vendor_approach']}")
    
    # Final recommendation
    best_option = recommendations['optimization_options'][0]
    print(f"\n🏆 RECOMMENDED ACTION:")
    if best_option['improvement'] > 5:
        print(f"   Implement '{best_option['scenario'].replace('_', ' ').title()}' scenario")
        print(f"   Expected improvement: {best_option['improvement']:.1f}% in cost efficiency")
    else:
        print("   Current plan is optimal. Proceed as planned.")

# Run the interactive system
print("\n🚀 Starting Interactive Decision Maker...")
interactive_decision_maker()


🚀 Starting Interactive Decision Maker...
🏗️ POWERGRID REAL-TIME DECISION MAKER

📝 Enter Project Details (or 'quit' to exit):


State:  Tamil Nadu
Tower Type (Type-A/Type-B/Type-C/Type-D):  Type-A
Substation Type (132kV/220kV/400kV/765kV):  132kV
Budget (Crores):  18.4
Tax Rate (%):  12
Month (1-12):  8
Geographic Risk (1-10):  6



🎯 DECISION ANALYSIS RESULTS

📊 PROJECT OVERVIEW:
   Location: Tamil Nadu
   Configuration: Type-A + 132kV
   Budget: 18.4 Cr | Risk: 6/10
   Timeline: Month 8

📦 MATERIAL PREDICTION:
   Base Demand: 393 tons
   Cost Efficiency: 21.34 tons/Cr

💡 OPTIMIZATION OPPORTUNITIES:
   1. Timing Optimized:
      Demand: 411 tons | Budget: 18.4 Cr
      Efficiency: 22.32 tons/Cr
      Improvement: -4.6%
   2. Risk Optimized:
      Demand: 391 tons | Budget: 18.4 Cr
      Efficiency: 21.27 tons/Cr
      Improvement: +0.3%
   3. Fully Optimized:
      Demand: 854 tons | Budget: 63.71 Cr
      Efficiency: 13.40 tons/Cr
      Improvement: +37.2%

⚠️  RISK ASSESSMENT:
   🔴 Monsoon Construction (Medium): Potential 30% delay in construction
      💡 Mitigation: Plan indoor pre-fabrication work
   🔴 Budget Constraint (High): Risk of material shortages
      💡 Mitigation: Consider phased procurement

🛒 PROCUREMENT STRATEGY:
   Recommended Quantity: 432 tons
   Safety Margin: 10.0%
   Timing: Can schedule p


🔍 Perform another analysis? (y/n):  n 


In [5]:
def save_decision_system(system, filename='powergrid_decision_system.pkl'):
    """Save the trained decision system"""
    joblib.dump(system, filename)
    print(f"💾 Decision system saved as {filename}")

def load_decision_system(filename='powergrid_decision_system.pkl'):
    """Load a trained decision system"""
    system = joblib.load(filename)
    print(f"📂 Decision system loaded from {filename}")
    return system

# Save the system for future use
save_decision_system(decision_system)

print("\n" + "=" * 60)
print("✅ REAL-TIME DECISION SYSTEM DEPLOYMENT READY!")
print("=" * 60)
print("\n🎯 Features Included:")
print("   • 🤖 AI-Powered Demand Prediction")
print("   • 💡 Optimization Scenarios")
print("   • ⚠️  Risk Assessment")
print("   • 🛒 Procurement Strategy")
print("   • 📊 Visual Dashboard")
print("   • 🔄 Real-time Interactive Interface")
print("   • 💾 Model Persistence")

print("\n🚀 Usage:")
print("   1. Run interactive_decision_maker() for live inputs")
print("   2. Use get_decision_recommendations() for API integration")
print("   3. Call create_decision_dashboard() for visual analysis")

💾 Decision system saved as powergrid_decision_system.pkl

✅ REAL-TIME DECISION SYSTEM DEPLOYMENT READY!

🎯 Features Included:
   • 🤖 AI-Powered Demand Prediction
   • 💡 Optimization Scenarios
   • ⚠️  Risk Assessment
   • 🛒 Procurement Strategy
   • 📊 Visual Dashboard
   • 🔄 Real-time Interactive Interface
   • 💾 Model Persistence

🚀 Usage:
   1. Run interactive_decision_maker() for live inputs
   2. Use get_decision_recommendations() for API integration
   3. Call create_decision_dashboard() for visual analysis
